In [ ]:
import os
import langchain

from langchain_core.documents import Document
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFLoader
import dotenv

dotenv_path = os.path.join(os.path.dirname('.env'))
if os.path.exists(dotenv_path):
    dotenv.load_dotenv(dotenv_path)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print('sucessfully loaded the env')

### create sample SQLLite database

```sql

In [ ]:
import sqlite3

os.makedirs('data/database', exist_ok=True)
conn = sqlite3.connect('data/database/example.db')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS departments')
c.execute('DROP TABLE IF EXISTS employees')
c.execute('''
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT,
    salary REAL NOT NULL,
    departId INTEGER,
    FOREIGN KEY (departId) REFERENCES departments(id)
)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS departments (
    id INTEGER PRIMARY KEY NOT NULL,
    name TEXT NOT NULL,
    description TEXT
)
''')

### Insert records into the Employees table and Departments table
    
```
```

In [ ]:
# INSERT INTO Employees (name, age, department_id) VALUES ('Alice', 30, 1);
# INSERT INTO Employees (name, age, department_id) VALUES ('Bob', 25, 2);
# INSERT INTO Employees (name, age, department_id) VALUES ('Charlie', 35, 1);
# INSERT INTO Employees (name, age, department_id) VALUES ('David', 28, 3);
# INSERT INTO Employees (name, age, department_id) VALUES ('Eve', 32, 2);
# INSERT INTO Departments (name) VALUES ('HR');
# INSERT INTO Departments (name) VALUES ('Engineering');
# INSERT INTO Departments (name) VALUES ('Marketing');

employees = [
    ('Alice', 'Senior software engineer with 5 years of experience in backend development.', 90000, 2),
    ('Bob', 'Marketing specialist with a focus on digital campaigns and social media.', 60000, 3),
    ('Charlie', 'HR manager with expertise in recruitment and employee relations.', 75000, 1),
    ('David', 'Junior developer with a passion for learning new technologies.', 50000, 2),
    ('Eve', 'Sales executive with a track record of exceeding targets.', 65000, 3)
]
departments = [
    (1,'HR', 'Handles recruitment, employee relations, and benefits.'),
    (2,'Engineering', 'Responsible for product development and maintenance.'),
    (3,'Marketing', 'Focuses on market research, advertising, and promotions.') 
]


In [ ]:
c.execute('DELETE FROM departments')
c.executemany('INSERT INTO departments (id, name, description) VALUES (?, ?, ?)', departments)
c.executemany('INSERT INTO employees (name, description, salary, departId) VALUES (?, ?, ?, ?)', employees)
conn.commit()

In [ ]:
c.execute('SELECT * FROM employees, departments WHERE employees.departId = departments.id')
conn.close()

### Database Content Process

In [ ]:
# Method 1: Using LangChain's SQLDatabase
from langchain_community.utilities import SQLDatabase

# Method 2: Using Langchain
from langchain.sql_database import SQLDatabase

# Usage example:
db = SQLDatabase.from_uri("sqlite:///data/database/example.db")

print(f' Tables are : {db.get_usable_table_names()}')
print(f' Table info : {db.get_table_info()}')




### Parse the data and create a Document List

In [ ]:
from typing import List
from langchain_core.documents import Document

## Iterate all tables and put in Document List
def sql_to_doc(db_path:str) -> List[Document]:
    db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
    table_names = db.get_usable_table_names()
    documents = []
    for table in table_names:
        query = f"SELECT * FROM {table}"
        result = db.run(query)
        content = f"Table: {table}\n{result}"
        print(f'\n{content}\n')
        documents.append(Document(page_content=content, metadata={"table": table}))
    return documents

documents = sql_to_doc('data/database/example.db')
print(f'Number of documents created from DB: {len(documents)}')
for doc in documents:
    print(doc.page_content)
    print('---')
    print(f'Metadata: {doc.metadata}')


# def sql_to_doc_v2(db_path:str) -> List[Document]:
#     db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
#     c = db.cursor()
#     c.execute("SELECT name FROM sqlite_master WHERE type='table';")
#     table_names = [row[0] for row in c.fetchall()]
#     documents = []
#     for table in table_names:
#         print(f'Table found: {table}')
#         c.execute(f"SELECT * FROM {table}")
#         rows = c.fetchall()
#         row_count = len(rows)
#         print(f'Number of rows in {table}: {row_count}')
#         documents.append(Document(page_content=str(rows), 
#                                       metadata={"table": table}))

#         for row in rows:
#             print(row)
           


    # documents = []
    # for table in table_names:
    #     query = f"SELECT * FROM {table}"
    #     result = db.run(query)
    #     content = f"Table: {table}\n{result}"
    #     print(f'\n{content}\n')
    #     documents.append(Document(page_content=content, metadata={"table": table}))
    # return documents



In [ ]:
import sqlite3
import json
from langchain_core.documents import Document

def sql_to_doc_dict(db_path: str):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_names = [row[0] for row in c.fetchall()]
    documents = []
    for table in table_names:
        print(f'Table found: {table}')
        c.execute(f"SELECT * FROM {table}")
        rows = c.fetchall()
        col_names = [desc[0] for desc in c.description]
        dict_rows = [dict(zip(col_names, row)) for row in rows]
        print(f'Number of rows in {table}: {len(dict_rows)}')
        # Convert list of dicts to JSON string for page_content
        documents.append(
            Document(page_content=json.dumps(dict_rows, indent=2),
                     metadata={"table": table, "source": db_path, "num_rows": len(dict_rows)})
        )
    conn.close()
    return documents

# Usage
documents = sql_to_doc_dict('data/database/example.db')
print(f'Number of documents created from DB: {len(documents)}')
for doc in documents:
    print(doc.page_content)
    print('---')
    print(f'Metadata: {doc.metadata}')